In [1]:
%matplotlib qt5
from process_files import *
from func_target import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [6]:
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
tgdir = config['targetplate']['directory']
settings = [f for f in os.listdir(tgdir) if 'settings' in f]
settings = tgdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
#
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(tgdir+os.sep+line))//3

In [24]:
tg_name = config['targetplate'][line+'/l0data'] 
hdul = pf.open(tg_name)
tg0 = hdul[0].data
tg1 = hdul[1].data
tg2 = hdul[2].data
aff = hdul[3].data
aff1 = aff[0]
aff2 = aff[1]

In [25]:
tg0_ = np.mean(tg0, axis=2)
tg1_ = np.mean(tg1, axis=2)
tg2_ = np.mean(tg2, axis=2)
blink_frames([tg0_, tg1_, tg2_], repeat=1)

In [113]:
sx1, sy1 = np.linalg.norm(aff1[:,0]), np.linalg.norm(aff1[:,1])
sx2, sy2 = np.linalg.norm(aff2[:,0]), np.linalg.norm(aff2[:,1])
mag = np.mean([sx1, sy1, sx2, sy2])
#
rotang1 = np.degrees(np.arctan2(-aff1[0,1]/sy1, aff1[0,0]/sx1))
rotang2 = np.degrees(np.arctan2(-aff2[0,1]/sy2, aff2[0,0]/sx2))
#
sh1_ = np.array([-2,-2])
sh2_ = np.array([-5,-1])
sh1 = aff1[::-1,2] + sh1_
sh2 = aff2[::-1,2] + sh2_
#
tg0_rot = tg0_
tg1_rot = rotate(tg1_, -rotang1, reshape=False)
tg2_rot = rotate(tg2_, -rotang2, reshape=False)
#
tg0_mag = tg0_rot
tg1_mag = zoom_clipped(tg1_rot, mag) 
tg2_mag = zoom_clipped(tg2_rot, mag)
#
tg0_shift = tg0_mag
tg1_shift = shift(tg1_mag, sh1)
tg2_shift = shift(tg2_mag, sh2)

In [115]:
roi = 800
tg0_plt = get_imcenter(tg0_shift, roi, roi)
tg1_plt = get_imcenter(tg1_shift, roi, roi)
tg2_plt = get_imcenter(tg2_shift, roi, roi)
tg0_plt = (tg0_plt-tg0_plt.min())/(tg0_plt.max()-tg0_plt.min())
tg1_plt = (tg1_plt-tg1_plt.min())/(tg1_plt.max()-tg1_plt.min())
tg2_plt = (tg2_plt-tg2_plt.min())/(tg2_plt.max()-tg2_plt.min())
blink_frames([tg0_plt, tg1_plt], repeat=5)
# plt.figure(); plt.imshow(tg0_shift)
# plt.figure(); plt.imshow(tg1_shift)
# plt.figure(); plt.imshow(tg2_shift)

In [82]:
roi = 900
tg0_ = get_imcenter(np.mean(tg0, axis=2), roi, roi)
tg1_ = get_imcenter(np.mean(tg1, axis=2), roi, roi)
tg2_ = get_imcenter(np.mean(tg2, axis=2), roi, roi)
blink_frames([tg0_, tg1_, tg2_], repeat=1)

In [86]:
def compute_angle1(img):
    """ 
    |   Compute the angle of the lines in the image of a grid
    |   Input : 2d image array
    |   Output : angle [0,90)
    """
    td = np.uint8(255*(img-img.min())/(img.max()-img.min()))
    nyy, nxx = td.shape
    plt.figure(); plt.imshow(td)
    td = cv2.Canny(td, 50, 150)
    td = cv2.dilate(td, np.ones([5,5], dtype=np.uint8))
    td = cv2.erode(td, np.ones([5,5], dtype=np.uint8))
    # plt.figure(); plt.imshow(td)
    lines = cv2.HoughLinesP(td, rho=1, theta=np.pi/180, threshold=nxx//4, lines=50, minLineLength=nxx//1.5, maxLineGap=8)
    for line in lines:
        for x1, y1, x2, y2 in line:
            plt.plot([x1,x2],[y1,y2],color='red')
    # compute slope of the lines
    angs = (lines[:,0,1]-lines[:,0,3])/(lines[:,0,0]-lines[:,0,2])
    angs = np.degrees(np.arctan(angs))
    angs[np.argwhere(angs<0)] += 90 
    ang = np.median(angs)
    # remove extremes
    try:
        extremes = np.argwhere(np.abs(angs-ang)>3).flatten()
        angs = list(angs)
        del angs[extremes]
        ang = np.median(np.array(angs))
    except: pass
    return ang


In [94]:
# Rotation
ang0 = compute_angle1(tg0_)
ang1 = compute_angle1(tg1_)
ang2 = compute_angle1(tg2_)
print('Angles: ', ang0, ang1, ang2)
rotang1 = ang1 - ang0
rotang2 = ang2 - ang0
rotang1 = 0
rotang2 = 0
tg0_rot = tg0_
tg1_rot = rotate(tg1_, rotang1, reshape=False)
tg2_rot = rotate(tg2_, rotang2, reshape=False)
print('Rotation angles are: ', rotang1, rotang2)
config['targetplate']['rotang1'] = rotang1
config['targetplate']['rotang2'] = rotang2
config.write()


Angles:  34.02166767656435 33.994945269224424 34.97774892350883
Rotation angles are:  0 0


In [96]:
blink_frames([tg0_, tg1_], repeat=10)

In [104]:
# Scaling
mag1 = compute_magnification(tg0_rot, tg1_rot, minmag=0.98)
mag2 = compute_magnification(tg0_rot, tg2_rot, maxmag=1.02)
mag = 0.5*(mag1+mag2)
mag = 0.98
print('Magnification: ', mag1, mag2)
tg0_mag = tg0_rot
tg1_mag = zoom_clipped(tg1_rot, mag) 
tg2_mag = zoom_clipped(tg2_rot, mag)
config['targetplate']['mag'] = mag
config.write()
#       

100%|██████████| 101/101 [00:10<00:00,  9.99it/s]


Magnification:  0.9908 0.992


In [103]:
blink_frames([tg0_mag, tg2_mag], pause=1, repeat=5)

In [ ]:

def compute_shift(img1, img2, roi=0.5):
    ny, nx = np.int16(roi*np.array(img1.shape))
    im1_cen = get_imcenter(img1, nx, ny)
    im1_int = fit_2d_plane(im1_cen, w=im1_cen)
    im2_cen = get_imcenter(img2, nx, ny)
    im2_int = fit_2d_plane(im2_cen, w=im2_cen)
    im1, im2 = im1_cen-im1_int, im2_cen-im2_int
    im1, im2 = (im1-im1.mean())/im1.std(), (im2-im2.mean())/im2.std()
    # Window
    hamm_0 = np.reshape(np.hamming(im1.shape[0]), (im1.shape[0], 1))
    # hamm_0[32:-32] = hamm_0[31]
    hamm_0 /= hamm_0.max()
    hamm_1 = np.reshape(np.hamming(im1.shape[1]), (1, im1.shape[1]))
    # hamm_1[32:-32] = hamm_1[31]
    hamm_1 /= hamm_1.max()
    w = hamm_0*hamm_1
    # Images
    im1, im2 = w*im1, w*im2
    fim1, fim2 = np.fft.fft2(im1), np.fft.fft2(im2)
    corr = np.fft.ifftshift(np.abs(np.fft.ifft2(fim1*np.conj(fim2))))
    half = np.array([ny//2, nx//2])
    sh = np.argwhere(corr==corr.max()).flatten()[0:2]-half
    return sh

In [ ]:
# Translation
sh1 = compute_shift(tg0_mag, tg1_mag, roi=0.6)
sh2 = compute_shift(tg0_mag, tg2_mag, roi=0.6)
print('Shifts: ', sh1, sh2)
tg0_shift = tg0_mag
tg1_shift = shift(tg1_mag, sh1)
tg2_shift = shift(tg2_mag, sh2)
config['targetplate']['shift1_y'] = sh1[0]
config['targetplate']['shift1_x'] = sh1[1]
config['targetplate']['shift2_y'] = sh2[0]
config['targetplate']['shift2_x'] = sh2[1]
config.write()


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(get)

In [ ]:
# Crop 
vnow, hnow = tg0_shift.shape[0:2]
vsize, hsize = tg0.shape[0:2]
vcrop, hcrop = (vnow-vsize)//2, (hnow-hsize)//2
tg0_crop = tg0_shift[vcrop:vcrop+vsize, hcrop:hcrop+hsize] 
tg1_crop = tg1_shift[vcrop:vcrop+vsize, hcrop:hcrop+hsize]
tg2_crop = tg2_shift[vcrop:vcrop+vsize, hcrop:hcrop+hsize]

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,6))
axs[0].imshow(tg0_crop)
axs[1].imshow(tg1_crop)
axs[2].imshow(tg2_crop)
plt.tight_layout()

In [ ]:
import cv2
# Initiate ORB detector
sift = cv2.SIFT_create()
# find the keypoints and descriptors with ORB
kpts0, descr0 = sift.detectAndCompute(np.uint8(255*tg0),None)
kpts1, descr1 = sift.detectAndCompute(np.uint8(255*tg1),None)
kpts2, descr2 = sift.detectAndCompute(np.uint8(255*tg2),None)
# create BFMatcher object
bf = cv2.BFMatcher()
matches1 = bf.knnMatch(descr0, descr1, k=2)
matches1_good = []
for m,n in matches1:
    if m.distance < 0.75*n.distance:
        matches1_good.append(m)
matches2 = bf.knnMatch(descr0, descr2, k=2)
matches2_good = []
for m,n in matches2:
    if m.distance < 0.75*n.distance:
        matches2_good.append(m)
# Draw first 10 matches.
matches1_img = cv2.drawMatches(tg0, kpts0, tg1, kpts1, matches1_good, None, flags=2)
plt.figure(); plt.imshow(matches1_img)
matches2_img = cv2.drawMatches(tg0, kpts0, tg2, kpts2, matches2_good, None, flags=2)
plt.figure(); plt.imshow(matches2_img)

In [ ]:
pts0 = np.array([kp.pt for kp in kpts0[0:10]], dtype=np.float32)
pts1 = np.array([kp.pt for kp in kpts1[0:10]], dtype=np.float32)
pts2 = np.array([kp.pt for kp in kpts2[0:10]], dtype=np.float32)
affine1 = cv2.estimateAffinePartial2D(pts0, pts1)[0]
affine2 = cv2.estimateAffinePartial2D(pts0, pts2)[0]

In [ ]:
scale1 = affine1[0,0]/affine1[1,1]
rotang1 = np.arctan2(affine1[1,0]/scale1, affine1[1,1]/scale1)
shift1 = affine1[:,2]
print(scale1, rotang1, shift1)
#
scale2= affine2[0,0]/affine2[1,1]
rotang2 = np.arctan2(affine2[1,0]/scale2, affine2[1,1]/scale2)
shift2 = affine2[:,2]
print(scale2, rotang2, shift2)